In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

image_path = r'/home/plorenc/Desktop/AiR_ISS/AVS/pedestrian/ROI.jpg'

img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
img_shape = img.shape
img_shape

(240, 360, 3)

In [4]:
def calculate_f1_score(thresh, I_ref):
    # Binaryzacja maski referencyjnej
    I_ref = cv2.threshold(I_ref, 127, 255, cv2.THRESH_BINARY)[1]

    # Obliczenie True Positives, False Positives, False Negatives
    TP = np.sum((thresh == 255) & (I_ref == 255))
    FP = np.sum((thresh == 255) & (I_ref == 0))
    FN = np.sum((thresh == 0) & (I_ref == 255))

    # Oblicz Precision i Recall
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0

    # Oblicz F1-score
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return (f1, precision, recall)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

kernel = np.ones((2, 2), np.uint8)

img_ = np.zeros(img_shape[:2]).astype(np.uint8)

f1_array = []

for i in range(300, 1100, 2):
    image_path = "/home/plorenc/Desktop/AiR_ISS/AVS/pedestrian/input/in%06d.jpg" % i  
    image_path_reference = "/home/plorenc/Desktop/AiR_ISS/AVS/pedestrian/groundtruth/gt%06d.png" % i  

    I_c = cv2.imread(image_path)
    I_ref = cv2.imread(image_path_reference)

    I_c = cv2.cvtColor(I_c, cv2.COLOR_BGR2RGB)

    I = cv2.cvtColor(I_c, cv2.COLOR_BGR2GRAY).astype(np.uint8)
    I_dif = cv2.absdiff(I, img_)
    img_ = I

    (T, thresh) = cv2.threshold(I_dif, 15, 255, cv2.THRESH_BINARY)  
    thresh = cv2.medianBlur(thresh, 5)
    thresh = cv2.erode(thresh, kernel=(8, 8), iterations=1)
    thresh = cv2.dilate(thresh, kernel=(12, 12), iterations=5)


    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh, connectivity=8)

    if stats.shape[0] > 1:
        # Wybieramy komponent o największym obszarze
        tab = stats[1:, 4]  # Kolumna 4 zawiera obszar
        pi = np.argmax(tab) + 1  # Indeks największego komponentu (dodajemy 1, aby uwzględnić tło)
        
        # Rysowanie prostokąta wokół największego obiektu
        x, y, w, h, area = stats[pi]
        cv2.rectangle(I_c, (x, y), (x + w, y + h), (155, 0, 0), 2)
        
        # Dodanie tekstu z polem obiektu
        cv2.putText(I_c, f"{area:.2f}", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0))
        # Dodanie tekstu z numerem obiektu
        cx, cy = centroids[pi]
        cv2.putText(I_c, f"{pi}", (int(cx), int(cy)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0))

    # Łączenie obrazów
    combined_image = cv2.hconcat([I_c, cv2.cvtColor(thresh, cv2.COLOR_GRAY2RGB), I_ref])
    f1 = calculate_f1_score(thresh, cv2.cvtColor(I_ref, cv2.COLOR_BGR2GRAY))
    f1_array.append(f1)
    cv2.imshow(f"IMG", combined_image)
    cv2.waitKey(10)

cv2.destroyAllWindows()


In [6]:
cv2.destroyAllWindows()

In [7]:
advg = 0
num = 0

for f in f1_array:
    if f[0] != 0:
        advg += f[0]
        num += 1

print(f"{advg/num*100:2f}")

71.619843
